In [12]:
from ddot import Ontology
from ddot.Ontology import parse_obo
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
%matplotlib inline
root_dir = '/home/hermuba/data0118/fig/'

# our ontology
ont2 = Ontology.from_table('/home/hermuba/data0118/network1122/candidate_ontologies')
# align to aro
align_aro = pd.read_csv('~/data0118/network1122/candidate_ontologies_term_anno.csv', header = 0, index_col = 0)
align_aro.set_index('index', inplace = True)

# load original gene annotation
gold_anno = pd.read_pickle('/home/hermuba/data0118/goldstandard/ec_rmplasmid_node_anno_df')

/home/hermuba/miniconda3/envs/ddot/lib/python3.6/site-packages/ddot/Ontology.py:1606: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  table = pd.read_table(table, comment='#', header=header)


In [19]:
onto_root = '/home/hermuba/data0118/ontologies/'
from ddot.Ontology import parse_obo
parse_obo(onto_root+'go-basic.obo', output_file = onto_root+'go.ddot', id2name_file = onto_root+'go_id2name.ddot', id2namespace_file = onto_root+'go_id2namespace.ddot')

# gene term mapping



In [39]:
geneterm = []
gt = gold_anno.loc[ont2.genes, 'GO']
for g in gt.index:
    if type(gt[g]) == set:
        for goterm in list(gt[g]):
            geneterm.append([goterm, g])
geneterm = pd.DataFrame(geneterm, columns = ['go_term', 'gene'])

In [40]:
go = Ontology.from_table(onto_root+'go.ddot', mapping = geneterm, mapping_parent = 'go_term', mapping_child = 'gene', propagate = None)

In [41]:
go

511 genes, 44702 terms, 1645 gene-term relations, 91525 term-term relations
node_attributes: []
edge_attributes: ['biological_process', 'is_a']

In [42]:
align_go = ont2.align(go)

collapse command: /home/hermuba/miniconda3/envs/ddot/lib/python3.6/site-packages/ddot/alignOntology/collapseRedundantNodes /tmp/tmpnyl_c705


/home/hermuba/miniconda3/envs/ddot/lib/python3.6/site-packages/ddot/Ontology.py:713: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  names=['Child', 'Parent'])


collapse command: /home/hermuba/miniconda3/envs/ddot/lib/python3.6/site-packages/ddot/alignOntology/collapseRedundantNodes /tmp/tmpju7y33qe
Alignment command: /home/hermuba/miniconda3/envs/ddot/lib/python3.6/site-packages/ddot/alignOntology/calculateFDRs /tmp/tmplzyaet8g /tmp/tmpbhs2qsj8 0.05 criss_cross /tmp/tmpxmytvjpq 100 40 gene


/home/hermuba/miniconda3/envs/ddot/lib/python3.6/site-packages/ddot/Ontology.py:150: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  header=None)


In [44]:
align_go.columns = [c+'_GO' for c in align_go.columns]


gold_anno

In [54]:
all_node = align_aro.merge(align_go, left_index = True, right_index = True).append(gold_anno.loc[ont2.genes])


In [58]:
# ndex server cannot take sets
all_node['domain'] = all_node['domain'].astype(str)
all_node['pathway'] = all_node['pathway'].astype(str)
all_node['GO'] = all_node['GO'].astype(str)
ont2.update_node_attr(all_node)

In [59]:
ndex_server = 'http://public.ndexbio.org'
ndex_user, ndex_pass = 'b101102109', '5mY87$747l'
url, _ = ont2.to_ndex(ndex_server=ndex_server, ndex_user=ndex_user, ndex_pass=ndex_pass,  layout="bubble-collect")
print(url)

http://public.ndexbio.org/v2/network/83637079-7f73-11ea-aaef-0ac135e8bacf
